# Dataset report: Overlap

In [11]:
import altair as alt
alt.data_transformers.disable_max_rows()
import numpy as np
import pandas as pd

import cytoxnet.dataprep.io
import cytoxnet.dataprep.featurize

### load all of the datasets, concatenate, and label sources

In [12]:
lunghini_fish = cytoxnet.dataprep.io.load_data('lunghini_fish_LC50')
lunghini_daphnia = cytoxnet.dataprep.io.load_data('lunghini_daphnia_EC50')
lunghini_algea = cytoxnet.dataprep.io.load_data('lunghini_algea_EC50')
zhu_rat = cytoxnet.dataprep.io.load_data('zhu_rat_LD50')
chembl_ecoli = cytoxnet.dataprep.io.load_data('chembl_ecoli_MIC')

In [13]:
sets_cleaned = []
locals_ = locals()
for set_name in ['lunghini_fish',
                 'lunghini_daphnia',
                 'lunghini_algea',
                 'zhu_rat',
                 'chembl_ecoli']:
    set_ = locals_[set_name]
    set_['source'] = set_name
    set_ = set_[['smiles', 'source']]
    sets_cleaned.append(set_)
dataframe = pd.concat(sets_cleaned, ignore_index=True)

In [14]:
dataframe

,smiles,source
0,C=C(Br)c1ccccc1,lunghini_fish
1,BrC(Br)Br,lunghini_fish
2,N#CC(Br)Br,lunghini_fish
3,BrC(Br)c1ccccc1C(Br)Br,lunghini_fish
4,BrC(Br)c1ccccc1OCC1CO1,lunghini_fish
...,...,...
18397,C#CCN(C)Cc1cc2cc(OCc3ccccc3)ccc2[nH]1.O=C(O)C(...,chembl_ecoli
18398,CC1CC(=O)Nc2c1c(Br)nc1nc3c(c(N)c21)CCCC3,chembl_ecoli
18399,COc1ccc(C(=N)Nc2cc(C(=O)Nc3nc(C(=O)NCCN4CCOCC4...,chembl_ecoli
18400,Cc1ccc2nc(Cl)c(/C=N/O)cc2c1,chembl_ecoli


### featurize the dataset (with the default Circular Fingerprint featurizer)


In [15]:
cytoxnet.dataprep.featurize.molstr_to_Mol(dataframe, 'smiles')
dataframe = cytoxnet.dataprep.featurize.add_features(dataframe)

## apply UMAP

In [16]:
import umap.umap_ as umap

In [17]:
dataframe['CircularFingerprint']

0        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1        [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2        [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3        [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4        [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                               ...                        
18397    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
18398    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
18399    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...
18400    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
18401    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: CircularFingerprint, Length: 18402, dtype: object

In [18]:
%%time
umap_model = umap.UMAP(metric = "jaccard",
                      n_neighbors = 25,
                      n_components = 2,
                      low_memory = False,
                      min_dist = 0.001)
X_umap = umap_model.fit_transform(np.vstack(dataframe['CircularFingerprint'].values))
dataframe["UMAP_0"], dataframe["UMAP_1"] = X_umap[:,0], X_umap[:,1]

/opt/anaconda3/envs/cytoxnet/lib/python3.7/site-packages/umap/umap_.py:1762: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  "inverse_transform will be unavailable".format(self.metric)
/opt/anaconda3/envs/cytoxnet/lib/python3.7/site-packages/pynndescent/pynndescent_.py:924: UserWarning: Failed to correctly find n_neighbors for some samples.Results may be less than ideal. Try re-running withdifferent parameters.
  "Failed to correctly find n_neighbors for some samples."
CPU times: user 1min 4s, sys: 1.44 s, total: 1min 5s
Wall time: 26.6 s


In [19]:
## remove non bitwise columns
dataset_ = dataframe[['UMAP_0', 'UMAP_1', 'source']]

**Create an interactive plot demostrating the overlap of molecule space between datasets.**

In [20]:
selection = alt.selection_multi(fields=['source'], bind='legend')
alt.Chart(dataset_).mark_circle(size=60).encode(
    x='UMAP_0',
    y='UMAP_1',
    color=alt.Color('source:N', scale=alt.Scale(scheme='category20b')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.01))
).add_selection(selection)



alt.Chart(...)

**We can see that the overlap between data is not very high and, in particular, the ChemBL Ecoli dataset occupies a more different molecule space in comparison to the other species. This sparsity is a concern for machine learning applications, particularly for multitask Random Forest analysis.**